In [1]:
import sys
from librosa.core import resample
import pandas as pd
import numpy as np
from IPython.display import Audio
import torch
import pathlib
def create_dir(filename):
    pathlib.Path('/'.join(filename.split('/')[:-1])).mkdir(parents=True, exist_ok=True)
from tqdm.notebook import tqdm
sys.path.append('Conv-TasNet/src/')
from conv_tasnet import *
from pit_criterion import cal_loss
from collections import Counter
import os
device = 0
device_ids = [0, 1, 2, 3]
root = './'
sr = 8000
torch.cuda.set_device(device)

# Pre-Train Baseline

In [8]:
tas2 = ConvTasNet.load_model('final.pth.tar').cuda(device)
tasnet = ConvTasNet(N = 256, L = 20, B = 256, H = 512, P = 3, X = 8, R = 4, C = 3, norm_type="gLN", causal=0,
             mask_nonlinear='relu').cuda(device)

own_state = tasnet.state_dict()
for name, param in tqdm(tas2.state_dict().items()):
    if name not in own_state:
         continue
    if isinstance(param, torch.nn.Parameter):
        param = param.data
    try:
        own_state[name].copy_(param)
    except:
        print('shape mismatch')

optimizer = torch.optim.Adam(tasnet.parameters(), lr = 0.0001)
tasnet = nn.DataParallel(tasnet, device_ids = device_ids)

if os.path.exists('models/tasnet.pth'):
    print('load model')
    checkpoint = torch.load('models/tasnet.pth')
    tasnet.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    loss = checkpoint['loss']
    
tasnet.train()
pass

shape mismatch

load model


In [9]:
class OverlayDataSet(torch.utils.data.Dataset):
    def __init__(self, csv, num_talker = 3):
        super().__init__()
        self.segments = pd.read_csv(root+csv)
        self.speakers = list(set(self.segments['speaker']))
        self.speakers.sort()
        self.spkr2idx = {spkr:i for i, spkr in enumerate(self.speakers)}
        self.num_talker = num_talker
        
    def __len__(self):
        return len(self.segments)
    def __getitem__(self, idx):
        talkers = []
        sigs = []
        for i in range(self.num_talker):
            if i!=0:
                idx = np.random.randint(len(self.segments))
            seg = self.segments.iloc[idx]
            while seg['speaker'] in talkers:
                idx = np.random.randint(len(self.segments))
                seg = self.segments.iloc[idx]
            sig = np.load(root+seg['segfile'])
            sig-=np.mean(sig)
            sig/=np.std(sig)
            sigs.append(sig)
            talkers.append(seg['speaker'])
        sigs = np.array(sigs)
        return np.sum(sigs, axis = 0), sigs


trainset = OverlayDataSet('files/train-segments.csv')
valset = OverlayDataSet('files/val-segments.csv')
testset = OverlayDataSet('files/test-segments.csv')
idx = np.random.randint(len(testset))

mixture, sources = testset[idx]
Audio(mixture, rate = sr)

In [10]:
Audio(sources[0], rate = sr)

In [11]:
Audio(sources[1], rate = sr)

In [12]:
Audio(sources[2], rate = sr)

In [14]:
batch_size = 32
trainloader  = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory = True, num_workers = 32)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, pin_memory = True, num_workers = 32)
tasnet.train()

for epoch in range(4, 64):
    running_loss = 0.0
    for batch_idx, (mixture, sources) in enumerate(tqdm(trainloader)):
        optimizer.zero_grad()
        mixture, sources = mixture.float().cuda(device), sources.float().cuda(device)
        out = tasnet(mixture)
        loss, max_snr, estimate_source, reorder_estimate_source = \
            cal_loss(sources, out, torch.ones(mixture.shape[0], dtype = torch.int32).cuda(device)*2*sr) # 2 seconds has 2*sr samples
        loss.backward()
        torch.nn.utils.clip_grad_norm_(tasnet.parameters(), 0.5)
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f ' % 
                  (epoch + 1, batch_idx + 1, running_loss / 200))
            running_loss = 0.0
            torch.save({
            'model_state_dict': tasnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, 'models/tasnet.pth')
    with torch.no_grad():
        running_loss = 0.0
        tasnet.eval()
        for batch_idx, (mixture, sources) in enumerate(tqdm(valloader)):
            mixture, sources = mixture.float().cuda(device), sources.float().cuda(device)
            out = tasnet(mixture)
            loss, max_snr, estimate_source, reorder_estimate_source = \
                cal_loss(sources, out, torch.ones(mixture.shape[0], dtype = torch.int32).cuda(device)*2*sr)

            running_loss += loss.item()*sources.shape[0]
        print(running_loss/len(valset))

[5,   200] loss: -7.928 
[5,   400] loss: -7.895 
[5,   600] loss: -7.682 
[5,   800] loss: -7.773 
[5,  1000] loss: -7.862 
[5,  1200] loss: -7.998 
[5,  1400] loss: -7.806 
[5,  1600] loss: -7.667 
[5,  1800] loss: -7.899 
[5,  2000] loss: -7.803 
[5,  2200] loss: -8.122 
[5,  2400] loss: -7.979 
[5,  2600] loss: -7.746 
[5,  2800] loss: -8.007 
[5,  3000] loss: -7.924 
[5,  3200] loss: -7.942 
[5,  3400] loss: -7.958 
[5,  3600] loss: -7.824 
[5,  3800] loss: -7.760 
[5,  4000] loss: -7.952 
[5,  4200] loss: -7.758 
[5,  4400] loss: -7.811 
[5,  4600] loss: -7.710 
[5,  4800] loss: -8.013 
[5,  5000] loss: -7.849 
[5,  5200] loss: -8.020 
[5,  5400] loss: -7.859 
[5,  5600] loss: -7.809 
[5,  5800] loss: -7.913 
[5,  6000] loss: -7.938 
[5,  6200] loss: -7.909 
[5,  6400] loss: -7.940 
[5,  6600] loss: -8.010 
[5,  6800] loss: -7.900 
[5,  7000] loss: -7.996 




-7.6879274742156865


[6,   200] loss: -7.976 
[6,   400] loss: -7.996 
[6,   600] loss: -7.765 
[6,   800] loss: -7.889 
[6,  1000] loss: -7.891 
[6,  1200] loss: -8.080 
[6,  1400] loss: -7.920 
[6,  1600] loss: -7.755 
[6,  1800] loss: -7.947 
[6,  2000] loss: -7.794 
[6,  2200] loss: -8.123 
[6,  2400] loss: -8.074 
[6,  2600] loss: -7.907 
[6,  2800] loss: -8.060 
[6,  3000] loss: -8.051 
[6,  3200] loss: -8.022 
[6,  3400] loss: -8.022 
[6,  3600] loss: -7.896 
[6,  3800] loss: -7.790 
[6,  4000] loss: -8.077 
[6,  4200] loss: -7.940 
[6,  4400] loss: -7.910 
[6,  4600] loss: -7.700 
[6,  4800] loss: -8.112 
[6,  5000] loss: -7.973 
[6,  5200] loss: -8.023 
[6,  5400] loss: -7.875 
[6,  5600] loss: -7.958 
[6,  5800] loss: -8.018 
[6,  6000] loss: -7.974 
[6,  6200] loss: -7.925 
[6,  6400] loss: -7.896 
[6,  6600] loss: -8.014 
[6,  6800] loss: -7.932 
[6,  7000] loss: -7.997 




-7.6902729885511665


[7,   200] loss: -7.996 
[7,   400] loss: -8.059 
[7,   600] loss: -7.815 
[7,   800] loss: -7.870 
[7,  1000] loss: -7.964 
[7,  1200] loss: -8.082 
[7,  1400] loss: -7.912 
[7,  1600] loss: -7.748 
[7,  1800] loss: -7.947 
[7,  2000] loss: -7.816 
[7,  2200] loss: -8.144 
[7,  2400] loss: -8.061 
[7,  2600] loss: -7.853 
[7,  2800] loss: -8.070 
[7,  3000] loss: -8.061 
[7,  3200] loss: -8.059 
[7,  3400] loss: -8.019 
[7,  3600] loss: -7.862 
[7,  3800] loss: -7.856 
[7,  4000] loss: -8.037 
[7,  4200] loss: -7.846 
[7,  4400] loss: -8.013 
[7,  4600] loss: -7.776 
[7,  4800] loss: -8.096 
[7,  5000] loss: -8.003 
[7,  5200] loss: -8.103 
[7,  5400] loss: -7.930 
[7,  5600] loss: -7.991 
[7,  5800] loss: -8.044 
[7,  6000] loss: -8.010 
[7,  6200] loss: -7.953 
[7,  6400] loss: -7.988 
[7,  6600] loss: -8.105 
[7,  6800] loss: -7.978 
[7,  7000] loss: -7.896 



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[26,  4400] loss: -8.343 
[26,  4600] loss: -8.123 


Traceback (most recent call last):


KeyboardInterrupt: 

  File "/home/junzhez2/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/junzhez2/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/junzhez2/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/junzhez2/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


# Maybe also try doing recursive defogging

In [19]:
torch.manual_seed(0)
np.random.seed(0)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, pin_memory = True, num_workers = 32)
with torch.no_grad():
    running_loss = 0.0
    tasnet.eval()
    for batch_idx, (mixture, sources) in enumerate(tqdm(testloader)):
        mixture, sources = mixture.float().cuda(device), sources.float().cuda(device)
        out = tasnet(mixture)
        loss, max_snr, estimate_source, reorder_estimate_source = \
            cal_loss(sources, out, torch.ones(mixture.shape[0], dtype = torch.int32).cuda(device)*2*sr)

        running_loss += loss.item()*sources.shape[0]
    print(running_loss/len(testset))


-8.075121239824593


In [18]:
# baseline TasNet SNR: -8.117dB
torch.save({
'model_state_dict': tasnet.module.state_dict(),
'optimizer_state_dict': optimizer.state_dict(),
'loss': loss
}, 'models/stable/tasnet-baseline.pth')